### Before start need load file to content folder

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
data_path = '/content/drive/My Drive/data/'

### Wraper for calculate time

In [0]:
import time

def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

## Pdf to Text - tika

install lib for works with pdf 

https://github.com/erfelipe/PDFtextExtraction


In [0]:
#pip install tika

  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=4972e0f41af7e46104737465d3b2341883e7585c892822c8f87ad91bf4e22e0f
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [0]:
from tika import parser

@timeit
def pdf_parser_tika(path):
  raw = parser.from_file(path)
  raw = str(raw)

  safe_text = raw.encode('utf-8', errors='ignore')

  # safe_text = str(safe_text).replace("\n", "").replace("\\", "")
  # print('--- safe text ---' )
  # print( safe_text )
  return safe_text


In [0]:
book_name = 'Richard_Templar-The_Rules_of_Work-EN.pdf'
path_to_book = data_path + book_name

In [49]:
content_str = pdf_parser_tika(path_to_book)

'pdf_parser_tika'  797.96 ms


In [50]:
print(content_str)

b'{\'metadata\': {\'Author\': \'Richard Templar\', \'Content-Type\': \'application/pdf\', \'X-Parsed-By\': [\'org.apache.tika.parser.DefaultParser\', \'org.apache.tika.parser.pdf.PDFParser\'], \'X-TIKA:content_handler\': \'ToTextContentHandler\', \'X-TIKA:embedded_depth\': \'0\', \'X-TIKA:parse_time_millis\': \'750\', \'access_permission:assemble_document\': \'true\', \'access_permission:can_modify\': \'true\', \'access_permission:can_print\': \'true\', \'access_permission:can_print_degraded\': \'true\', \'access_permission:extract_content\': \'true\', \'access_permission:extract_for_accessibility\': \'true\', \'access_permission:fill_in_form\': \'true\', \'access_permission:modify_annotations\': \'true\', \'creator\': \'Richard Templar\', \'dc:creator\': \'Richard Templar\', \'dc:format\': \'application/pdf; version=1.6\', \'dc:title\': \'The Rules of Work\', \'meta:author\': \'Richard Templar\', \'pdf:PDFVersion\': \'1.6\', \'pdf:charsPerPage\': [\'0\', \'83\', \'2114\', \'2286\', \'

In [51]:
import ast
dict_str = content_str.decode("UTF-8")
content_pdf = ast.literal_eval(dict_str)

print(content_pdf)
title = content_pdf['metadata']['title']
content = content_pdf['content']

{'metadata': {'Author': 'Richard Templar', 'Content-Type': 'application/pdf', 'X-Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'X-TIKA:embedded_depth': '0', 'X-TIKA:parse_time_millis': '750', 'access_permission:assemble_document': 'true', 'access_permission:can_modify': 'true', 'access_permission:can_print': 'true', 'access_permission:can_print_degraded': 'true', 'access_permission:extract_content': 'true', 'access_permission:extract_for_accessibility': 'true', 'access_permission:fill_in_form': 'true', 'access_permission:modify_annotations': 'true', 'creator': 'Richard Templar', 'dc:creator': 'Richard Templar', 'dc:format': 'application/pdf; version=1.6', 'dc:title': 'The Rules of Work', 'meta:author': 'Richard Templar', 'pdf:PDFVersion': '1.6', 'pdf:charsPerPage': ['0', '83', '2114', '2286', '2809', '2560', '2265', '471', '2079', '2116', '1533', '1452', '1709', '1729', '1666', '1538', '81

In [53]:
title

'The Rules of Work'

In [0]:
file_name = data_path + title + '.txt'
with open(file_name, 'w+') as f:
    f.write(content)

## Pdf to Text - PyPDF2

In [0]:
#pip install PyPDF2

     |████████████████████████████████| 81kB 2.1MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=a0741b2a0b9f9103ff51fe5f3f7c1a47d76a9860c2da81cb23b16c0e7e28fe6d
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [0]:
from PyPDF2 import PdfFileReader

@timeit
def pdf_parser_PyPDF2(pdf_file):
    pdfreader = PdfFileReader(pdf_file)
    num_pages = pdfreader.numPages
    count = 0
    text = ""

    while count < num_pages:
        pageObj = pdfreader.getPage(count)
        count += 1
        text += pageObj.extractText()

    if text != "":
        text = text
    else:
        return None
        # text = textract.process(pdf_file, method='tesseract', language='eng')
    return text

In [0]:
content_pdf = pdf_parser_PyPDF2(file_path)

'pdf_parser_PyPDF2'  13067.07 ms
